In [14]:
import torch
from torch.utils.data import Dataset, DataLoader
import cv2
from glob import glob
import albumentations as A
from albumentations.pytorch import ToTensorV2
import pandas as pd
from tqdm import tqdm
import os

class SamplingDataset(Dataset) :
    def __init__(self, files, root_path, transform) :
        self.files = files
        self.transform = transform
        self.root_path = root_path
        
    def __len__(self) :
        return len(self.files)
    
    def __getitem__(self, idx) :
        
        file_path = self.files[idx] 
        
        img = cv2.imread("../data/test/"+file_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        transform_img = self.transform(image=img)['image']
        
        transform_img = transform_img.transpose(2, 0, 1)
#         img = img.transpose(2,0,1)
        
#         transform_img = torch.tensor(transform_img, dtype=torch.float)
#         img = torch.tensor(img, dtype=torch.float) / 255.0
        
        return transform_img / 255


class Mean_Std() :
    def __init__ (self, dataloader, img_output_pos=0) :
        self.mean, self.std = self.get_val(dataloader, pos=img_output_pos)
        
    def get_val(self, dataloader, pos=0) :
        RGB_CHANNEL = 3
        img_mean = torch.zeros(3)
        img_std = torch.zeros(3)
        for idx, output in enumerate(tqdm(dataloader)) :
            for i in range(RGB_CHANNEL):
                if not pos is None :
                    img_mean[i] += output[pos][:, i, :, :].mean()
                    img_std[i] += output[pos][:, i, :, :].std()
                else : 
                    img_mean[i] += output[:, i, :, :].mean()
                    img_std[i] += output[:, i, :, :].std()
    
                
        return img_mean.div(len(dataloader)), img_std.div(len(dataloader))


transforms = A.Compose([
    A.Resize(224, 224),
#     ToTensorV2()
])

train_df = pd.read_csv('../data/test_df.csv',error_bad_lines=False)
display(train_df)
img_files = list(train_df['file_name'])

sample_dataset = SamplingDataset(img_files, None,transforms)
dataloader = DataLoader(sample_dataset, batch_size=128, shuffle=True)



C:\Users\sub\AppData\Local\Temp\ipykernel_1620\693361015.py:63: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  train_df = pd.read_csv('../data/test_df.csv',error_bad_lines=False)


,index,file_name
0,0,20000.png
1,1,20001.png
2,2,20002.png
3,3,20003.png
4,4,20004.png
...,...,...
2149,2149,22149.png
2150,2150,22150.png
2151,2151,22151.png
2152,2152,22152.png


In [15]:
M_S = Mean_Std(dataloader, None)
MM = M_S.mean / 255
SS = M_S.std / 255

print(MM)
print(M_S.mean)
print(SS)
print(M_S.std)

100%|██████████████████████████████████████████████████████████████████████████████████| 17/17 [00:31<00:00,  1.84s/it]

tensor([0.0016, 0.0015, 0.0015])
tensor([0.4178, 0.3926, 0.3861])
tensor([0.0010, 0.0010, 0.0010])
tensor([0.2601, 0.2582, 0.2559])
